# Which product characteristics (Features) drive customer sentiment (winpercent) ?
> Key result expected: Best feature extraction

In [1]:
# Load necessary packages
from sklearn.model_selection import train_test_split # for Lasso algorithm's feature extraction
import pandas as pd # dataframe manipulation
import matplotlib.pyplot as plt # plotting graphs
import numpy as np # efficient calculation
import statsmodels.formula.api as smf # Linear regression: In order to use "formula notation" similar to R
from sklearn.linear_model import Lasso # # Lasso regression to extract key features

## Import and check the data quality

In [2]:
# Read CSV as DataFrame df
df = pd.read_csv('candy-data.csv') # Path to the input file
# Glimpse of data
print(df.head())

  competitorname  chocolate  fruity  caramel  peanutyalmondy  nougat  \
0      100 Grand          1       0        1               0       0   
1   3 Musketeers          1       0        0               0       1   
2       One dime          0       0        0               0       0   
3    One quarter          0       0        0               0       0   
4      Air Heads          0       1        0               0       0   

   crispedricewafer  hard  bar  pluribus  sugarpercent  pricepercent  \
0                 1     0    1         0         0.732         0.860   
1                 0     0    1         0         0.604         0.511   
2                 0     0    0         0         0.011         0.116   
3                 0     0    0         0         0.011         0.511   
4                 0     0    0         0         0.906         0.511   

   winpercent  
0   66.971725  
1   67.602936  
2   32.261086  
3   46.116505  
4   52.341465  


In [3]:
# Inspect data types
print(df.dtypes) # exlcuding competitorname, all other features are numerical

competitorname       object
chocolate             int64
fruity                int64
caramel               int64
peanutyalmondy        int64
nougat                int64
crispedricewafer      int64
hard                  int64
bar                   int64
pluribus              int64
sugarpercent        float64
pricepercent        float64
winpercent          float64
dtype: object


In [4]:
# Check for missing values
print(df.describe()) # No missing values 
# Since all input values are between ~0 to ~1, no standardization required

       chocolate     fruity    caramel  peanutyalmondy     nougat  \
count  85.000000  85.000000  85.000000       85.000000  85.000000   
mean    0.435294   0.447059   0.164706        0.164706   0.082353   
std     0.498738   0.500140   0.373116        0.373116   0.276533   
min     0.000000   0.000000   0.000000        0.000000   0.000000   
25%     0.000000   0.000000   0.000000        0.000000   0.000000   
50%     0.000000   0.000000   0.000000        0.000000   0.000000   
75%     1.000000   1.000000   0.000000        0.000000   0.000000   
max     1.000000   1.000000   1.000000        1.000000   1.000000   

       crispedricewafer       hard        bar   pluribus  sugarpercent  \
count         85.000000  85.000000  85.000000  85.000000     85.000000   
mean           0.082353   0.176471   0.247059   0.517647      0.478647   
std            0.276533   0.383482   0.433861   0.502654      0.282778   
min            0.000000   0.000000   0.000000   0.000000      0.011000   
25%     

In [5]:
# How the composition is distributed ?
df.iloc[:,1:7].sum(axis=0,numeric_only=True) # Seems mostly candies include chocolate and fruity

chocolate           37
fruity              38
caramel             14
peanutyalmondy      14
nougat               7
crispedricewafer     7
dtype: int64

## Visual EDA for correlations

In [6]:
# Colormap to know the correlation between ingredients
plt.figure(figsize=(16,16))
plt.suptitle("Correlation of candy features", fontsize= 16)
sns.heatmap(df.corr(), annot=True, fmt="0.2f",cmap="YlGnBu", vmin=-1.0, vmax=1.0) # anchor the colormap
plt.tight_layout(pad=5.0)
plt.show()

NameError: name 'sns' is not defined

<Figure size 1152x1152 with 0 Axes>

## Interpretation:
#### Examining last row of the colormap
> 1. Chocholate having correlation factor:0.64 seems to be highly correlated with winpercent
> 2. Bar with correlation factor:0.43 is also correlated next to chocolate
> 3. Peanutyalmondy also can be influencing factor

## Let's examine these 3 ingredients in detail

In [ ]:
# Since there are more than 1 time, we run this piece, let's build function
def viz_impact(df,col_x): 
    # plots violin and pie graphs for a given ingredient comparing it's impact on
    # winpercent,pricepercentile,sugarpercentile
    viz_graph = plt.figure(figsize=(20,20))
    viz_graph.suptitle("Impact by ingredient: "+ col_x,fontsize= 40)

    plt.subplot(2,2,1)
    plt.title("Influence on Winning precentage",fontsize=16)
    sns.violinplot(x= col_x, y="winpercent", data=df,color="brown")
    plt.xlabel("Does it have ? (Yes = 1, No = 0)")

    plt.subplot(2,2,2)
    plt.title("Influence on Price percentage",fontsize=16)
    sns.violinplot(x=col_x,y="pricepercent", data=df)
    plt.xlabel("Does it have ? (Yes = 1, No = 0)")

    if(col_x != "bar"): # Since bar wrapping has no influence on sugarpercentile
        plt.subplot(2,2,3)
        plt.title("Total percentage of Candies with and without this ingredient",fontsize=16)
        plt.pie(df[col_x].value_counts(), autopct='%1.1f%%',
                colors=["#EEDD82","#98FB98"],
                labels=["Without it","With it"],
                shadow=True)

    plt.subplot(2,2,4)
    plt.title("Influence on Sugar percentage",fontsize=16)
    sns.violinplot(x=col_x, y="sugarpercent", data=df)
    plt.xlabel("Does it have ? (Yes = 1, No = 0)")


    plt.show()

In [ ]:
viz_impact(df,"chocolate")

## Interpretation
> 1. With chocolate, the winpercent shoots up.
> 2. However, with chocolate, price also raises.
> 3. Sugar composition seems to be compensated by other ingredients.
> 4. More Candies had chocolate then without it.

In [ ]:
# Impact fro next composition
viz_impact(df,"peanutyalmondy")

## Interpretation:
> 1. With peanutyalmondy, the winpercent also shoots up.
> 2. However, peanutyalmondy is very costly.
> 3. Sugar composition seems to be moderately distributed around half the range.
> 4. Mostly, Candies are without peanutyalmondy.

In [ ]:
# Impact from bar
viz_impact(df,"bar")

## Interpretation:
> 1. With bar, winpercent seems to better (around 60%).
> 2. However, with bar, it becomes costly
> 3. Candies with and without bar have moderate distribution
> 4. Mostly, Candies are usually without bar.

## Having explored the data, in order to more concreately understand the influence of these composition, Let us apply regression methods:
* Linear regression
* Lasso regression

## Linear regression

In [ ]:
formula = 'winpercent ~ chocolate + fruity + caramel + peanutyalmondy + nougat + crispedricewafer + hard + bar + pluribus + sugarpercent + pricepercent'
# create a fitted model with all three features
lm = smf.ols(formula=formula, data = df).fit()
lm.summary()#lm.params

## Interpretation:
> 1. since p values for chocolate,peanutyalmondy and fruity are significant (<0.05), can reject null hypothesis for those and the rest cannot
> 2. chocolate,peanutyalmondy and fruity is highly associated with winpercent
    * If chocolate is included, on an average, winpercent increase by 19.8 factor
    * If peanutyalmondy is included, on an average, winpercent increase by 10.1 factor
    * If fruity is included, on an average, winpercent increase by 9.4 factor
    * If candy is wrapped as bar, on an average, winpercent very slightly increase by 0.44 factor
> 3. Although pricepercent is negatively correlated, since pvalue is above .05, this cant be justified statistically

## 2. Lasso regression to extract features

In [ ]:
print(df.competitorname.unique().shape)
# Since the focus is on composition and competitorname is unique for each row, 
# there is no info from it therefore,it can be excluded
# Also, winpercent is the target and hence can be excluded
df_columns = df.drop(['winpercent','competitorname'], axis=1).columns

In [ ]:
values

In [ ]:
X = df.drop(['winpercent','competitorname'], axis=1).values
y = df.winpercent.values
# Instantiate a lasso regressor: lasso
lasso = Lasso(alpha = .4, normalize = True)

# Fit the regressor to the data
lasso.fit(X,y)

# Compute and print the coefficients
lasso_coef = lasso.fit(X,y).coef_
print(lasso_coef)

# Plot the coefficients
plt.plot(range(len(df_columns)), lasso_coef)
plt.xticks(range(len(df_columns)), df_columns.values, rotation=75)
plt.margins(0.02)
plt.show()

## Interpretation:
> 1. Most vital feature is chocolate
> 2. Next important is peanutyalmondy
> 3. Rest features seems to be insignificant as per lasso

## *Conclusion*
> 1. Key product characteristics that drive customer sentiment is Chocolate and Peanutyalmondy
> 2. New product be wrapped preferably as bar and could focus mainly on these three ingredients: 
    * Chocolate
    * Peanutyalmondy
    * Fruity